In [1]:
import cadquery as cq
from jupyter_cadquery import (
    open_viewer,
    set_defaults,
    show
)
from jupyter_cadquery.replay import replay, enable_replay
set_defaults(axes=True, timeit=False, axes0=True)
cv = open_viewer("Examples", cad_width=640, height=400, glass=True)
enable_replay(False, False)
show_object = replay

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


In [2]:
# inputs
length = 420
egde_width = 18
off_center = 6 # move the pockets off center (to the side) this amount
breadth = 54  # Breadth of the half the final part

# tuple of shaft diameter and handle diameter (at the lowest point of the handle)
flat = (
    (3, 14),
    (3.5, 16),
    (4, 18),
    (5, 20),
    (6, 22),
    (7, 25),
)
ph_and_pz = (
    (6, 22 + 0.4),
    (5, 18 + 0.4),
    (4, 14),
    (5, 18),
    (6, 22),
    (8, 28 + 0.6),
)
hex_ = (
    (2.5, 12.5),
    (2.5, 12.5),
    (2.8, 12.5),
    (3.3, 14),
    (4.5, 17),
    (5.5, 18 + 0.4),
    (6.7, 20 + 0.8),
)
torx = (
    (3, 14),
    (3.5, 14),
    (3.5, 16.5),
    (4, 18),
    (5, 20),
    (6, 22),
)

screw_drivers_a = [*flat, *ph_and_pz]
screw_drivers_b = [*reversed(hex_), *torx]


In [3]:
def slot_sketch(w, d=50, z=0):
    return (
        cq.Sketch()
        .arc((0, 0), w / 2, 0, 180.0)
        .segment((-w / 2, -d))
        .segment((w / 2, -d))
        .close()
        .assemble()
        .moved(cq.Location(cq.Vector(0, 0, z)))
        .edges()
    )


def cut_out_for_one_screwdriver(handle, shaft):
    handle_slot = handle + 1.4
    slot_width_top = shaft + 4
    slot_width_bot = shaft + 0.5

    result = (
        cq.Workplane()
        .placeSketch(
            slot_sketch(slot_width_top, z=0),
            slot_sketch(slot_width_bot, z=-15),
            slot_sketch(slot_width_bot, z=-200),
        )
        .loft(ruled=True)
        .faces(">Z")
        .wires()
        .toPending()
        .extrude(30)
        .moveTo()
        .workplane(offset=-3.0)
        .circle(slot_width_bot / 2)
        .workplane(offset=handle_slot / 4)
        .circle(handle_slot / 2 - 2)
        .workplane(offset=0.4)
        .circle(handle_slot / 2 - 0.4)
        .workplane(offset=0.5)
        .circle(handle_slot / 2)
        .workplane(offset=5.0)
        .circle(handle_slot / 2)
        .workplane(offset=40.0)
        .circle(handle_slot / 2 + 15)
        .loft(ruled=True)
    ).translate((0, 0, -16))
    return result
show(cut_out_for_one_screwdriver(handle=16, shaft=4)).export_png("images/cut_out.png")

![images/cut_out.png](images/cut_out.png)

In [4]:

def handle_to_spacing(handle):
    return (handle + 4) * 1.5

def combine_pockets(inputs):
    x_offset = egde_width
    remainder_length = (
        length
        - egde_width * 2
        - sum(handle_to_spacing(handle) for _, handle in inputs)
    )
    pockets = cq.Workplane("XY")
    for shaft, handle in inputs:
        block_width = handle_to_spacing(handle) + remainder_length / len(inputs)
        pocket = cut_out_for_one_screwdriver(handle, shaft)
        pockets = pockets.union(pocket.translate((x_offset + block_width / 2, 0, 0)))
        x_offset += block_width  # Adjust the offset for the next pocket
    return pockets

pockets_a = combine_pockets(screw_drivers_a)
pockets_b = combine_pockets(screw_drivers_b)
show(pockets_b).export_png("images/pockets_b.png")

![images/pockets_b.png](images/pockets_b.png)

In [5]:
# these are the pockets for the poles supporting the final holder
pole = (
    cq.Workplane("XY")
    .transformed(rotate=(-8, 0, 0))
    .circle(6.7)
    .workplane(offset=-16)
    .circle(7.5)
    .loft()
    .faces("<Z")
    .wires()
    .toPending()
    .extrude(-150)
)

def generate_holder(block):
    # Set the dimensions for the design
    height = 40  # Height of the object
    offset = 4  # Offset value for positioning
    outer_fillet = 30  # Fillet radius on the under / outer edge
    inner_fillet = 15 # Fillet radius on the top / inner edge
    # Start constructing the object
    result = (
        cq.Workplane("XY")
        .move(length / 2, offset)  # Positioning the workplane
        .rect(length, breadth)  # Creating a rectangle on the workplane
        .extrude(-height)  # Extruding the rectangle to create a 3D shape
        .cut(
            block.rotate((0, 0, 0), (10, 0, 0), 7)
        )  # Cutting the shape with a rotated block
        .edges("<Y and (not >Z) and (not <Z)")  # Selecting specific edges
        .fillet(5)
    )  # Applying a fillet of radius 5 to the selected edges

    # Further modifications to the object
    result = (
        result.faces("<X")  # Selecting the face on the -X side
        .workplane()  # Creating a new workplane on the selected face
        .move(
            breadth / 2 - offset, -height + outer_fillet
        )  # Positioning on the new workplane
        .vLine(-outer_fillet)  # Drawing a vertical line
        .hLine(-outer_fillet)  # Drawing a horizontal line
        .close()  # Closing the sketch to form a loop
        .cutThruAll()
    )  # Cutting through the entire object along the sketch

    for point in (
        result.faces(">Z")
        .workplane(centerOption="CenterOfBoundBox", offset=5)
        .rarray(length - 21, breadth, 2, 1)
    ).vals():
        result = result.cut(pole.translate(point))

    result = (
        result.faces("<X")  # Selecting the face on the -X side
        .workplane()  # Creating a new workplane on the selected face
        .move(-breadth / 2 - 4, -inner_fillet)
        .vLine(inner_fillet)  # Drawing a vertical line
        .hLine(inner_fillet)
        .close()  # Closing the sketch to form a loop
        .cutThruAll()  # Cutting through the entire object along the sketch
    )

    result = result.translate((0, -breadth / 2 - 4, 0))
    return result


result = generate_holder(pockets_a)
show(result).export_png("images/result_a.png")

![images/result_a.png](images/result_a.png)

In [6]:
result = generate_holder(pockets_a)
result = result.mirror(result.faces(">Y")).union(generate_holder(pockets_b))
result
show(result).export_png("images/result_combined.png")

![result_combined.png](result_combined.png)

In [7]:
# Cut part in two halves using dovetail joints so it fits on my printer
# the paramaters below are a bit janky, but they work for me
d = 0.2  # cut width (will influence the fit, depends strongly on printer and settings)
v = -6.5  # move cut line on "north" side of the part so it will not intersect a screwdriver pocket
w = 5.3  # move cut line on "south" side of the part so it will not intersect a screwdriver pocket
e = 3.6  # 1/4 width of the dovetail
f = 5  # depth of the dovetail
cut_height = 100
# the following two lines are a very specific correction for a bug in (I presume) the cadkernel.
random_number_for_cadkernel_bug = 4  # If zero, the sweep completely blows up
silly_correction_because_of_cadkernel_bug = 0.32  # the cut should go through the center, but somehow it does not. This is a hack to fix that.
path = (
    cq.Workplane("XY")
    .center(length / 2, random_number_for_cadkernel_bug)
    .moveTo(v, breadth+2)
    .lineTo(v, 5 * e)
    .line(f, e)
    .line(0, -4 * e)
    .line(-f, e)
    .lineTo(v, 0)
    .lineTo(w, 0)
    .line(0, -3 * e)
    .line(-f, e)
    .line(0, -4 * e)
    .line(f, e)
    .lineTo(w, -breadth-2)
    .wire()
)
path

dovetail_cut = (
    cq.Workplane("XZ")
    .workplane(offset=random_number_for_cadkernel_bug)
    .center(length / 2, 0)
    .rect(d, cut_height)
    .sweep(path)
    .translate(
        (
            0,
            - random_number_for_cadkernel_bug
            + silly_correction_because_of_cadkernel_bug,
            0,
        )
    )
)
show(dovetail_cut).export_png("images/dovetail_cut.png")

![images/dovetail_cut.png](images/dovetail_cut.png)

In [8]:
# cut out the dovetail cut, splitting the part in two solids
result = result.cut(dovetail_cut)
show(result).export_png("images/screw-driver-rack-split.png")

![images/screw-driver-rack-split.png](images/screw-driver-rack-split.png)

In [9]:
# flip parts to enable print without infill or support
part_a = result.solids(">X").rotate((0, 0, 0), (0, 1, 0), 180)
part_b = result.solids("<X").rotate((0, 0, 0), (0, 1, 0), 180)

cq.exporters.export(part_a, "screw-driver-rack-a.step")
show(part_a).export_png("images/screw-driver-rack-a.png")

![images/screw-driver-rack-a.png](images/screw-driver-rack-a.png)

In [10]:
cq.exporters.export(part_b, "screw-driver-rack-b.step")
show(part_b).export_png("images/screw-driver-rack-b.png")

![images/screw-driver-rack-b.png](images/screw-driver-rack-b.png)